Ce code a été fait pendant les nuits de l'IA à Bordeaux



In [7]:
import math 
import numpy as np
 
# our sigmoid function, tanh is a little nicer than the standard 1/(1+e^-x)
def sigmoid(x):
    return np.tanh(x)
 
# derivative of our sigmoid function
def dsigmoid(x):
    return 1.0 - x**2
     
     
class MLP:
    def __init__(self, *args):
        self.args = args
        n = len(args)
         
        self.layers = [np.ones(args[i] + (i==0)) for i in range(0, n)]
         
        self.weights = list()
        for i in range(n-1):
            R = np.random.random((self.layers[i].size, self.layers[i+1].size))
            self.weights.append((2*R-1)*0.20) # On initialize les poids dans [-0.2,0.2)
             
        self.m = [0 for i in range(len(self.weights))]
             
         
    def predict(self, inputs):
        self.layers[0][:-1] = inputs
         
        for i in range(1, len(self.layers)):
            self.layers[i] = sigmoid(np.dot(self.layers[i-1], self.weights[i-1]))
             
        return np.round(self.layers[-1])
         
    def backPropagate(self, inputs, correctOutputs, a=0.1, m=0.1):
         
        error = correctOutputs - self.predict(inputs)
        de = error * dsigmoid(self.layers[-1])
        deltas = list()
        deltas.append(de)
         
        for i in range(len(self.layers)-2, 0, -1):
 
            deh = np.dot(deltas[-1], self.weights[i].T) * dsigmoid(self.layers[i])
            deltas.append(deh)
             
        deltas.reverse()
         
        for i, j in enumerate(self.weights):
             
            layer = np.atleast_2d(self.layers[i])
            delta = np.atleast_2d(deltas[i])
             
            dw = np.dot(layer.T,delta)
            self.weights[i] += a*dw + m*self.m[i]
            self.m[i] = dw
            
    def learn(self, inputs, correctResult):
             self.backPropagate( inputs, correctResult)
 
 
 
pat = (((0, 0, 0, 0),(0, 0, 0, 0, 0, 0, 0, 0, 0, 0)),
    ((1, 0, 0, 0),(0, 1, 0, 0, 0, 0, 0, 0, 0, 0)),
    ((1, 0, 1, 0),(0, 0, 1, 0, 0, 0, 0, 0, 0, 0)),
    ((0, 1, 1, 0),(0, 0, 0, 1, 0, 0, 0, 0, 0, 0)),
    ((0, 0, 1, 0),(0, 0, 0, 0, 1, 0, 0, 0, 0, 0)),
    ((0, 1, 0, 1),(0, 0, 0, 0, 0, 1, 0, 0, 0, 0)),
    ((1, 0, 0, 1),(0, 0, 0, 0, 0, 0, 1, 0, 0, 0)),
    ((1, 1, 0, 0),(0, 0, 0, 0, 0, 0, 0, 1, 0, 0)),
    ((0, 0, 1, 1),(0, 0, 0, 0, 0, 0, 0, 0, 1, 0)),
    ((0, 1, 0, 0),(0, 0, 0, 0, 0, 0, 0, 0, 0, 1)))    

 
 
n = MLP(4, 3, 10)

In [8]:
# --- Test avant entrainement
for p in pat:
    print( p[0], "=>", n.predict(p[0]))

(0, 0, 0, 0) => [ 0.  0. -0. -0.  0.  0.  0.  0. -0. -0.]
(1, 0, 0, 0) => [ 0.  0. -0. -0.  0.  0.  0.  0. -0. -0.]
(1, 0, 1, 0) => [ 0.  0.  0.  0.  0.  0.  0. -0. -0. -0.]
(0, 1, 1, 0) => [ 0. -0. -0.  0.  0.  0.  0.  0. -0. -0.]
(0, 0, 1, 0) => [ 0.  0.  0. -0.  0.  0.  0. -0. -0. -0.]
(0, 1, 0, 1) => [ 0. -0. -0.  0. -0. -0. -0.  0. -0. -0.]
(1, 0, 0, 1) => [ 0. -0. -0.  0. -0. -0. -0.  0. -0. -0.]
(1, 1, 0, 0) => [ 0. -0. -0.  0. -0. -0. -0.  0. -0. -0.]
(0, 0, 1, 1) => [ 0. -0. -0.  0.  0.  0.  0.  0. -0. -0.]
(0, 1, 0, 0) => [ 0. -0. -0.  0. -0. -0. -0.  0. -0. -0.]


In [9]:
# --- Phase d'entrainement
print(n.weights)

for i in range(1000):
    for p in pat:
        n.learn(p[0], p[1])
         
print(n.weights)

[array([[ 0.02173961,  0.05195645, -0.0452688 ],
       [ 0.07361075,  0.07929892, -0.1892377 ],
       [-0.14373878, -0.1580865 ,  0.15674964],
       [ 0.00652301, -0.01444811, -0.19941808],
       [ 0.03060356, -0.16259148,  0.03974297]]), array([[-0.0081784 ,  0.16047309,  0.00727066, -0.12051123, -0.04766845,
        -0.17370336, -0.05520819,  0.05746444, -0.07831678,  0.14200646],
       [-0.17791351,  0.01401809,  0.0191687 , -0.03803886, -0.08614389,
        -0.12704051, -0.14856211, -0.10086125,  0.1877033 ,  0.0792308 ],
       [-0.05537953,  0.15187383,  0.04307381, -0.13826675,  0.15398306,
         0.03780507,  0.199595  , -0.15084086,  0.13869038,  0.14280448]])]
[array([[-0.27245975, -0.72880298, -0.30300789],
       [ 0.06407833, -1.37068183, -0.93409037],
       [-0.80858765,  0.66658637,  0.64113328],
       [-0.78792121, -0.28051883, -0.76878133],
       [ 0.4705727 ,  0.08874905,  0.9397658 ]]), array([[-0.0081784 ,  0.29509861, -0.25412278, -0.09935774, -0.14484193

In [10]:
# --- Test avant entrainement
for p in pat:
    print( p[0], "=>", n.predict(p[0]))

(0, 0, 0, 0) => [-0.  0.  0.  0.  0. -0. -0. -0. -0.  0.]
(1, 0, 0, 0) => [ 0.  1.  0.  0.  0.  0.  0.  0. -0.  0.]
(1, 0, 1, 0) => [-0.  0.  1.  0.  0. -0.  0. -0.  0. -0.]
(0, 1, 1, 0) => [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
(0, 0, 1, 0) => [-0.  0.  0.  0.  1. -0.  0. -0.  0. -0.]
(0, 1, 0, 1) => [ 0. -0. -0.  0. -0.  1.  0.  0.  0.  0.]
(1, 0, 0, 1) => [ 0.  0.  0.  0. -0.  0.  1.  0.  0.  0.]
(1, 1, 0, 0) => [ 0.  0. -0.  0. -0.  0.  0.  1. -0.  0.]
(0, 0, 1, 1) => [-0. -0.  0.  0.  0. -0.  0. -0.  1. -0.]
(0, 1, 0, 0) => [ 0.  0.  0.  0. -0.  0.  0.  0. -0.  1.]


In [11]:
print( (0,1,0,1), "=>", n.predict((0,1,0,1)))

(0, 1, 0, 1) => [ 0. -0. -0.  0. -0.  1.  0.  0.  0.  0.]
